In [17]:
!pip install pathpy

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [18]:
import pathpy as pp

In [19]:
#getting all the transactions related to a contract
import requests 
import json

contract = '0x7d56485e026d5d3881f778e99969d2b1f90c50af'
url = "https://api.etherscan.io/api?module=account&action=txlist&address="+contract+"&startblock=0&endblock=99999999&sort=asc&apikey=Y8TKAQ6SUHFFG1ND2UMMBN4XAYUT3JY7XV"
res = requests.get(url)
transactions = json.loads(res.text)['result']
size = len(transactions)
print('There are %d transactions with this contract.' % (size))

# this list contains the same transactions as the 'transactions' list, but 
# only keeping the relevant keys
newTransactions = []

# list of keys that we want to keep
relevantKeys = ['timeStamp', 'from', 'value', 'isError']
for transaction in transactions:
    newTrans = {myKey: transaction[myKey] for myKey in relevantKeys}
    newTransactions.append(newTrans)

# keep only the transactions that have not produced an error
newTransactions = [tran for tran in newTransactions if int(tran['isError']) == 0]

# order the newTransactions list based on timestamp
newTransactions = sorted(newTransactions, key=lambda k : k['timeStamp'])

# subtract the initial timestamp-1 from all the other timestamps, so 
# time starts from 1
# firstTimeStamp = int(newTransactions[0]['timeStamp']) - 1
# for transaction in newTransactions:
#     transaction['timeStamp'] = int(transaction['timeStamp']) - firstTimeStamp

# SUGGESTION: We can just make the time stamps 1,2,3, etc.
timestamp = 1
for transaction in newTransactions:
    transaction['timeStamp'] = timestamp
    timestamp += 1

# convert Wei to Ether
weiPerOneEther = 1000000000000000000
for transaction in newTransactions:
    weiValue = float(transaction['value'])
    etherValue = weiValue/weiPerOneEther
    transaction['value'] = round(etherValue,2)

# SUGGESTION: For the final demo, we can even convert the Ether to Euros,
# in order to make it more relatable

for transaction in newTransactions:
    print(transaction, end='\n')

There are 554 transactions with this contract.
{'timeStamp': 1, 'from': '0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e', 'value': 0.0, 'isError': '0'}
{'timeStamp': 2, 'from': '0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e', 'value': 0.0, 'isError': '0'}
{'timeStamp': 3, 'from': '0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e', 'value': 0.0, 'isError': '0'}
{'timeStamp': 4, 'from': '0x65c407ffea9fcff194fe9d3335d2b78416226056', 'value': 30.0, 'isError': '0'}
{'timeStamp': 5, 'from': '0x9f451c2b960638c7e083a2cf7ce081aab1a4108a', 'value': 1.0, 'isError': '0'}
{'timeStamp': 6, 'from': '0x16f53e55ed93efa0f025ae5df7200aa88ecd4fa6', 'value': 20.0, 'isError': '0'}
{'timeStamp': 7, 'from': '0x8fee195d9bc0129dc4faeef96342bf2e07860367', 'value': 18.0, 'isError': '0'}
{'timeStamp': 8, 'from': '0xf4bd49d7782f9398ac0428ed1468ac8d0a2ef322', 'value': 11.0, 'isError': '0'}
{'timeStamp': 9, 'from': '0x9f451c2b960638c7e083a2cf7ce081aab1a4108a', 'value': 30.0, 'isError': '0'}
{'timeStamp': 10, 'from': '0x9f451c2b96

In [20]:
# make a dictionary that maps all the wallets that interact with the contract
# to nice labels (W1, W2, etc.)
wallets = []
for transaction in newTransactions:
    wallet = transaction['from']
    if wallet not in wallets:
        wallets.append(transaction['from'])

walletsMap = {}
index = 0
for wallet in wallets:
    walletsMap[wallet] = 'W' + str(index)
    index += 1

print(walletsMap)


{'0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e': 'W0', '0x65c407ffea9fcff194fe9d3335d2b78416226056': 'W1', '0x9f451c2b960638c7e083a2cf7ce081aab1a4108a': 'W2', '0x16f53e55ed93efa0f025ae5df7200aa88ecd4fa6': 'W3', '0x8fee195d9bc0129dc4faeef96342bf2e07860367': 'W4', '0xf4bd49d7782f9398ac0428ed1468ac8d0a2ef322': 'W5', '0x9bf53a22130bc32e6062dfff490072d984acffea': 'W6', '0x6d32bc8ae9dfff3570bc1009d9f68f76ad7f2ff2': 'W7', '0x8e9934f68f921f1a86c1556e01bd9e4423852253': 'W8', '0x8db2e041b218b468a84660f7670f27ac12b6066c': 'W9', '0xdd098bb908985d9f9ef7029da4b1c9b7953334b6': 'W10', '0xcd17c2db51d52823d95df5b561187e1e1a66183e': 'W11', '0x3ae10a9ffb028be0a867a85896556c04c6e09f5a': 'W12', '0xbb689812b1b9033461a51a4ce27a47909aa0a01f': 'W13', '0xb3e3429824d31ba87edea73fac823a67f44c6015': 'W14', '0xe97e15a717e2694229277ea3394f194ad9f8414a': 'W15', '0x436cec39f266a3e0f81767267a5c730a2fbca9be': 'W16', '0x1f4ccee385f54f42d72eb4892ba7ac72653c9589': 'W17', '0x251e01cf6a27d0e5b80ed8ce2d59965a19366f15': 'W18', '0

In [21]:
# build the temporal network
network = pp.TemporalNetwork()
for transaction in newTransactions:
    timestamp = int(transaction['timeStamp'])
    wallet = transaction['from']
    walletLabel = walletsMap[wallet]
    network.add_edge('C', walletLabel ,timestamp)
style = {    
  'ts_per_frame': 1, 
  'ms_per_frame': 200,
  'look_ahead': 0, 
  'look_behind': 300, 
  'node_size': 20, 
  'inactive_edge_width':2,
  'active_edge_width': 4, 
  'active_node_color': "#FF69B4",
  'active_edge_color': "#FF69B4",
  'label_color' : '#ffffff',
  'label_size' : '10px',
  'label_offset': [0,5],
  'width':800,
  'height':600,
  # 'plot_higher_order_nodes': True

  }
pp.visualisation.plot(network, **style)
pp.visualisation.export_html(network, filename='contractVisualization.html', **style)

In [15]:
from collections import Counter
import pandas as pd
# create a dataframe from the list of dictionaries 
transaction_df= pd.DataFrame(newTransactions) 

# for each wallet label (W1,W2,etc.): its address (found in the walletsMap), 
# the number of transactions with contract
# and the total amount of Eth put into the contract

# walletsMap
#{'0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e': 'W0',
# '0x65c407ffea9fcff194fe9d3335d2b78yz416226056': 'W1',}

try:
    new_transaction_dict = transaction_df.drop('isError', axis=1,)
    transaction_df = pd.DataFrame.from_dict(new_transaction_dict)
except KeyError:
    transaction_df = pd.DataFrame.from_dict(transaction_df)

#
# initialize empty dataframe to merge the records
sample_record = ['wallet_id','example','address','example','number_of_transactions',0,'money_deposited', 0]
sample_record_dict = {sample_record[i]: [sample_record[i+1]] for i in range(0,len(sample_record), 2)}
wallet_info_df = pd.DataFrame.from_dict(sample_record_dict)

#initialize a list to append the values

# print(transaction_df['from'].value_counts())
occurences_per_address_pd_series = transaction_df['from'].value_counts()
# print('occurences_per_address\n',occurences_per_address_pd_series)
# sum value count depending on the from entity
for i, occurences in enumerate(occurences_per_address_pd_series):
    # get the address
    from_value = occurences_per_address_pd_series.index[i]
    # print('from_value', from_value)
    #get the number of occurences
    # print('occurences:', occurences)
    sum_amount_added = transaction_df.loc[transaction_df['from'] == from_value]['value'].sum()
    # print('sum_amount_added', sum_amount_added)
    wallet_id = walletsMap[from_value]
    # print('wallet_id:', wallet_id)
        # initialize new record to reorder columns in the predefined way
    only_values = [wallet_id, from_value, occurences, sum_amount_added] 
    wallet_info_df.loc[len(wallet_info_df)] = only_values
wallet_info_ready_df = wallet_info_df[1:]
# write it to html 
wallet_info_html = wallet_info_ready_df.to_html()
#write html to file
wallet_info_txt = open("wallet_info.html", "w")
wallet_info_txt.write(wallet_info_html)
wallet_info_txt.close()

     timeStamp                                        from  value
0            1  0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e   0.00
1            2  0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e   0.00
2            3  0x4464b8a559ed286c3ae2d20acd6ddf664f5c8c1e   0.00
3            4  0x65c407ffea9fcff194fe9d3335d2b78416226056  30.00
4            5  0x9f451c2b960638c7e083a2cf7ce081aab1a4108a   1.00
..         ...                                         ...    ...
485        486  0x61b0ac2075de46b75ed44d64568ac2d6281c56ba   0.01
486        487  0x48988836ca2ce04c3a8cd7736251fda393ecb0e0   0.10
487        488  0xfeeb9cbde35a1948387eaee41f6ed781bb0c8478   0.50
488        489  0x80028f80c7d5959c3eaf45a95bf3a1a0724352f6   0.01
489        490  0x57d10e88ebedb4ca0396fbf734d83b0ad27de925   0.01

[490 rows x 3 columns]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1f0ff243-a6be-405e-85de-a67e1fff1f5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>